# Stock Comparison Tool

Trying to implement something similar to https://www.barchart.com/myBarchart/quotes/SPY/interactive-chart but in ~~Gradio~~ Plotly + Dash.

The actual app is run by Gunicorn via the `app.py` script. This notebook is used for interactive development and testing.

In [1]:
from src.dash_app import NormalizedAssetPricesApp

if __name__ == "__main__":
    app = NormalizedAssetPricesApp()
    app.run(host="0.0.0.0", port=8050, debug=True)